In [75]:
import numpy as np
import cv2
import os
import re
import xlsxwriter

In [76]:
#global variables

ssd_results_dictionary = {}
ground_truth_dictionary = {}
treshold = 0.5
path_to_ground_truth = "./TestGround/"
path_to_images = "./TestImages/"
path_to_SSD_GT_results = "./SSD_GT_Outputs/"

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbikes", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

The function compares two rectangles by calculating the ratio of their intersection and the total area.

In [77]:
def intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

The next function step by step compares the coincidence of the coordinates obtained from the Ground Truth and the coordinates received from the Single Shot Detector. Comparison is made using the "intersection_over_union" function. The comparison results are stored in a special dictionary, which is used to make the final verdict.

In [78]:
def confusion_matrix(GTdictionary, SSDictionary, category):
    TP = 0
    FP = 0
    FN = 0
    iouresults = {}
    max_check = treshold
    
    #if SSD not get right category at all
    if not category in SSDictionary:
        FN = int(len(GTdictionary[category])/4)
        TP = 0
        FP = 0
        return TP,FP,FN
    
    #for each object in ground truth looking for most suitable SSD object.
    for inc in range(0, len(GTdictionary[category]), 4):
        for jnc in range(0, len(SSDictionary[category]), 4):
            #gain intersection over union results to determine
            iou = intersection_over_union(GTdictionary[category][inc:inc+4],SSDictionary[category][jnc:jnc+4])
            #for current GT object finding the most suitable SSD object
            #and write it number to result dictionary where key is the number of the GT object
            #if nothing matches the GT object with the given precision, then nothing will be written to the result dictionary
            if max_check < iou:
                max_check = iou
                iouresults[inc/4+1] = jnc/4+1
        #print("The iou check dict",iouresults)
        max_check = treshold
    
    #Only correct matches is written to the results dictionary, so it length is the number on correctly determined objects
    TP = len(iouresults)
    #remaining objects from GT dictionary are undefined by SSD
    FN = int(len(GTdictionary[category])/4)-len(iouresults)
    #objects from SSD dictionary that not muth the GT objects are determined as false detection
    FP = int(len(SSDictionary[category])/4)-len(dict.fromkeys(iouresults.values()))
    iouresults.clear()
    return TP,FP,FN

The following function is designed to control the operation of the main function "confusion_matrix". If something does not work as it should, this function will signal that the output does not match the predicted one.

In [ ]:
def test_confusion_matrix():
    #GIVEN
    GroundTruthDictionary = {"person":[225, 46, 273, 192,1,12,100,150]}
    SingleShotDetectorDictionary = {"person":[235, 56, 273, 181,281, 42, 328, 195]}
    #WHEN
    TruePositive,FalsePositive,FalseNegative = confusion_matrix(GroundTruthDictionary, SingleShotDetectorDictionary, "person")
    
    #THEN
    assert TruePositive == 1, "TruePositive must be 1"
    assert FalsePositive == 1, "TruePositive must be 1"
    assert FalseNegative == 1, "TruePositive must be 1"
    
test_confusion_matrix()

The function opens a file of a certain format with Ground Truth information, reads the coordinates of the objects and writes to the dictionary.

In [79]:
def get_coordinates_from_ground_truth(file, dictionary):
    groundFile = open(file, "r") #open file
    for line in groundFile:
        if line.find("Original label for object") == 0:
            separated = line.split()
            category_type = re.sub('"','', separated[-1]) #get type of object from txt file
        if line.find("Bounding box for object") == 0:
            separated = line.split()
            #get Top Left, Right Bottom coordinates from file
            coordinates = [int(re.sub('[^0-9]','', separated[12])),int(re.sub('[^0-9]','', separated[13])),int(re.sub('[^0-9]','',separated[15])),int(re.sub('[^0-9]','', separated[16]))]
            #save coordinates in the right dictionary element
            if category_type in dictionary:
                dictionary[category_type] += coordinates
            else:
                dictionary[category_type] = coordinates
    groundFile.close()

In [80]:
def write_frame_to_picture(image,source_path,save_path, dictionary,confidence,name_modificator):
    color = (255, 255, 0)
    image_processed = cv2.imread(source_path+image)
    for x_key in dictionary.keys():
        for inc in range(0, len(dictionary[x_key]), 4):
            startX, startY, endX, endY = dictionary[x_key][inc:inc+4]
            label = "{}: {:2}%".format(x_key, confidence * 100)
            cv2.rectangle(image_processed, (startX, startY), (endX, endY), color, 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image_processed, label, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    print(image.split(".")[0]+name_modificator+"."+image.split(".")[1])
    cv2.imwrite(save_path+image.split(".")[0]+name_modificator+"."+image.split(".")[1], image_processed)

In [81]:
print("[INFO] loading model…")
# 
conf_threshold = 0.5
net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt", "MobileNetSSD_deploy.caffemodel")


[INFO] loading model…


In [82]:
#initialize the entry in xlsx file
row = 0
col = 0
#create xlsx file to store results
workbook = xlsxwriter.Workbook('./Mega_test_results.xlsx')
worksheet = workbook.add_worksheet()
#prepare column names
worksheet.write(row, col,     "File name")
worksheet.write(row, col + 1, "Category")
worksheet.write(row, col + 2, "TP")
worksheet.write(row, col + 3, "FP")
worksheet.write(row, col + 4, "FN")
row += 1

In [83]:
#iterate over images and search for objects

# create folder to store images with detector frames.
if not os.path.exists(path_to_SSD_GT_results):
    os.makedirs(path_to_SSD_GT_results)

image_list = os.listdir(path_to_images)
image_list.sort() 
    
for image_file in image_list:
    print (image_file)
    worksheet.write(row, col, image_file)
    image = cv2.imread(path_to_images+image_file)
    image_GT = image
    #image = cv2.imread("TestImages/0001.png")
    (h, w) = image.shape[:2]
    #print("Image size",h, w)
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
    #print("[INFO] computing object detections…")
    net.setInput(blob)
    detections = net.forward()
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            #print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)
            
            #print ("type:",CLASSES[idx],"Top left X:Y",startX,":",startY,"Bottom right X:Y",endX,":",endY)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLORS[idx], 2)
                        
            if CLASSES[idx] in ssd_results_dictionary:
                ssd_results_dictionary[CLASSES[idx]] += [startX,startY,endX,endY]
            else:
                ssd_results_dictionary[CLASSES[idx]] = [startX,startY,endX,endY]
    
    image_ground_truth = image_file.split(".")[0]+".txt"
    get_coordinates_from_ground_truth(path_to_ground_truth+image_ground_truth,ground_truth_dictionary)
    for x_key in ssd_results_dictionary.keys():
        print("   SSD category:",x_key,"Values:",ssd_results_dictionary[x_key])
    
      
    for x_key in ground_truth_dictionary.keys():
        print("Ground category:",x_key,"Values:",ground_truth_dictionary[x_key])
    write_frame_to_picture(image_file,path_to_images,path_to_SSD_GT_results, ground_truth_dictionary,1.0,"_GT")
    
    for x_cat in ground_truth_dictionary.keys():
        TruePositive,FalsePositive,FalseNegative = confusion_matrix(ground_truth_dictionary,ssd_results_dictionary,x_cat)
        print("For category '",x_cat,"' True Positive:",TruePositive,"False Negative:",FalseNegative,"False Positive:",FalsePositive)
        worksheet.write(row, col + 1, x_cat)
        worksheet.write(row, col + 2, TruePositive)
        worksheet.write(row, col + 3, FalsePositive)
        worksheet.write(row, col + 4, FalseNegative)
        row += 1
    ground_truth_dictionary.clear()
    ssd_results_dictionary.clear()
            #cv2.putText(image, label, (startX, y),
            #cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
    cv2.imwrite(path_to_SSD_GT_results+image_file.split(".")[0]+"_SSD."+image_file.split(".")[1], image)
    
    
    
    
    #cv2.imshow("Output", image)
    #cv2.waitKey(0)
    
#closing xlsx file
workbook.close()
    

carsgraz_001.png
   SSD category: car Values: [31, 36, 456, 275]
Ground category: car Values: [30, 42, 455, 286]
carsgraz_001_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_003.png
   SSD category: car Values: [355, 91, 630, 201]
Ground category: car Values: [349, 96, 628, 205]
carsgraz_003_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_004.png
   SSD category: car Values: [488, 114, 622, 230]
Ground category: car Values: [490, 111, 626, 222]
carsgraz_004_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_005.png
   SSD category: car Values: [493, 265, 619, 347]
Ground category: car Values: [497, 264, 617, 350]
carsgraz_005_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_006.png
   SSD category: car Values: [43, 51, 350, 196]
Ground category: car Values: [117, 52, 340, 199]
carsgraz_006_GT.png
For category ' car ' True Po

   SSD category: car Values: [-1, 75, 285, 275]
Ground category: car Values: [1, 72, 286, 275]
carsgraz_054_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_055.png
   SSD category: car Values: [396, 116, 603, 254]
Ground category: car Values: [399, 115, 598, 253]
carsgraz_055_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_058.png
   SSD category: car Values: [205, 126, 641, 359]
Ground category: car Values: [200, 124, 640, 364]
carsgraz_058_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_063.png
   SSD category: car Values: [298, 268, 642, 390]
Ground category: car Values: [304, 272, 640, 391]
carsgraz_063_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_065.png
   SSD category: car Values: [0, 81, 296, 242]
Ground category: car Values: [1, 79, 297, 241]
carsgraz_065_GT.png
For category ' car ' True Positive: 1 False Neg

   SSD category: car Values: [222, 65, 549, 196]
Ground category: car Values: [232, 64, 548, 198]
carsgraz_125_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_126.png
   SSD category: car Values: [390, 153, 559, 294]
Ground category: car Values: [385, 157, 566, 294]
carsgraz_126_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_127.png
   SSD category: car Values: [367, 80, 635, 312]
Ground category: car Values: [430, 139, 630, 318]
carsgraz_127_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_132.png
Ground category: car Values: [262, 201, 309, 239]
carsgraz_132_GT.png
For category ' car ' True Positive: 0 False Negative: 1 False Positive: 0
carsgraz_133.png
   SSD category: car Values: [235, 128, 578, 324]
Ground category: car Values: [243, 131, 578, 325]
carsgraz_133_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_134.pn

   SSD category: car Values: [182, 124, 355, 189]
Ground category: car Values: [225, 137, 348, 176]
carsgraz_204_GT.png
For category ' car ' True Positive: 0 False Negative: 1 False Positive: 1
carsgraz_205.png
Ground category: car Values: [127, 125, 216, 163]
carsgraz_205_GT.png
For category ' car ' True Positive: 0 False Negative: 1 False Positive: 0
carsgraz_206.png
   SSD category: car Values: [201, 87, 320, 150]
Ground category: car Values: [215, 94, 321, 142]
carsgraz_206_GT.png
For category ' car ' True Positive: 1 False Negative: 0 False Positive: 0
carsgraz_207.png
Ground category: car Values: [164, 103, 277, 164]
carsgraz_207_GT.png
For category ' car ' True Positive: 0 False Negative: 1 False Positive: 0
carsgraz_208.png
Ground category: car Values: [171, 186, 268, 244]
carsgraz_208_GT.png
For category ' car ' True Positive: 0 False Negative: 1 False Positive: 0
carsgraz_209.png
   SSD category: car Values: [340, 68, 587, 159]
Ground category: car Values: [345, 83, 583, 159]

   SSD category: pottedplant Values: [197, 78, 340, 312]
Ground category: person Values: [191, 110, 236, 213]
person_039_GT.png
For category ' person ' True Positive: 0 False Negative: 1 False Positive: 0
person_041.png
   SSD category: person Values: [127, 292, 171, 428]
Ground category: person Values: [125, 277, 179, 425]
person_041_GT.png
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_042.png
   SSD category: person Values: [150, 190, 221, 392]
Ground category: person Values: [140, 189, 226, 401]
person_042_GT.png
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_043.png
Ground category: person Values: [539, 127, 570, 248]
person_043_GT.png
For category ' person ' True Positive: 0 False Negative: 1 False Positive: 0
person_045.png
   SSD category: person Values: [87, 94, 464, 640]
Ground category: person Values: [88, 98, 465, 640]
person_045_GT.png
For category ' person ' True Positive: 1 False Negative: 0 False 

   SSD category: person Values: [55, 81, 332, 629]
Ground category: person Values: [58, 87, 349, 640]
person_082_GT.png
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_083.png
   SSD category: person Values: [83, 81, 443, 642]
Ground category: person Values: [80, 74, 446, 640]
person_083_GT.png
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_084.png
   SSD category: person Values: [11, 142, 405, 643]
Ground category: person Values: [11, 141, 424, 640]
person_084_GT.png
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_086.png
   SSD category: person Values: [68, 34, 489, 474]
Ground category: person Values: [50, 24, 467, 480]
person_086_GT.png
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_087.png
   SSD category: person Values: [34, 35, 631, 475]
Ground category: person Values: [58, 45, 618, 480]
person_087_GT.png
For category ' person ' True P

   SSD category: person Values: [235, 140, 422, 564, 136, 181, 220, 493]
Ground category: person Values: [145, 175, 226, 502, 233, 143, 381, 566]
person_124_GT.png
For category ' person ' True Positive: 2 False Negative: 0 False Positive: 0
person_128.png
   SSD category: person Values: [174, 176, 307, 496, 117, 184, 157, 294, 439, 186, 475, 406]
Ground category: person Values: [22, 191, 61, 286, 112, 185, 159, 292, 179, 178, 309, 495, 361, 183, 408, 294]
person_128_GT.png
For category ' person ' True Positive: 2 False Negative: 2 False Positive: 1
person_130.png
   SSD category: person Values: [253, 132, 390, 538, 99, 117, 228, 524]
   SSD category: pottedplant Values: [0, 24, 124, 371]
Ground category: person Values: [107, 128, 224, 525, 256, 133, 394, 539]
person_130_GT.png
For category ' person ' True Positive: 2 False Negative: 0 False Positive: 0
person_131.png
   SSD category: person Values: [175, 103, 343, 566]
Ground category: person Values: [176, 122, 332, 568]
person_131_GT.

   SSD category: person Values: [380, 56, 544, 459, 206, 73, 335, 430, 45, 16, 183, 334]
Ground category: person Values: [86, 20, 180, 367, 208, 70, 333, 441, 389, 48, 532, 451]
person_216_GT.png
For category ' person ' True Positive: 3 False Negative: 0 False Positive: 0
person_217.png
   SSD category: person Values: [157, 143, 449, 482, 505, 180, 643, 473]
Ground category: person Values: [166, 140, 436, 480, 519, 174, 640, 480]
person_217_GT.png
For category ' person ' True Positive: 2 False Negative: 0 False Positive: 0
person_223.png
   SSD category: person Values: [355, 71, 494, 438, 201, 109, 332, 445]
Ground category: person Values: [196, 96, 334, 451, 360, 78, 498, 432]
person_223_GT.png
For category ' person ' True Positive: 2 False Negative: 0 False Positive: 0
person_225.png
   SSD category: person Values: [340, 100, 460, 423, 177, 120, 296, 418, 89, 98, 121, 188]
Ground category: person Values: [79, 86, 127, 192, 182, 128, 304, 419, 295, 96, 328, 170, 344, 91, 461, 429]
per